In [ ]:
!tar -zxvf tr.tar.gz

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/token-classification/run_ner.py

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install datasets

In [ ]:
!mv train.csv train
!mv test.csv test
!mv dev.csv dev


In [ ]:
import csv
import json

In [ ]:
def convert_to_csv(filename):
  with open(filename, "r") as f:
   data = f.readlines()
  
  #data = filter(lambda x : x != "SAMPLE_START\tO\n", data)
  #data = list(data)
  print(data[0:24])
  for item in data[0:40]:
   print(item.split())
  
  
  with open(filename + '.json', 'w') as csvfile:
    words = []
    ner = []
    for item in data:

      item = item.split()
      if not item or item[0] == "[SEP]":
          d = {}
          d["words"] = words
          d["ner"] = ner
          csvfile.write(json.dumps(d) + "\n")
          words = []
          ner = []
      else:
        words.append(item[0])
        ner.append(item[1])


In [ ]:
def convert_to_csv_updated(filename):
  with open(filename, "r") as f:
   data = f.readlines()
  
  #data = filter(lambda x : x != "SAMPLE_START\tO\n", data)
  #data = list(data)
  print(data[0:24])
  for item in data[0:40]:
   print(item.split())
  
  
  with open(filename + '.json', 'w') as csvfile:
    words = []
    ner = []
    for item in data:

      item = item.split()
      if not item or item[0] == "[SEP]":
          d = {}
          d["words"] = words
          d["ner"] = []
          csvfile.write(json.dumps(d) + "\n")
          words = []
          ner = []
      else:
        words.append(item[0])
        #ner.append(item[1])


In [ ]:

convert_to_csv("pr-train.txt")
#convert_to_csv("test")
#convert_to_csv("dev")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_json("pr-train.txt.json", lines=True)

train, test = train_test_split(df, test_size=0.2, random_state=42)

train.to_json("train.json", lines=True, orient='records')
test.to_json("test.json", lines=True, orient='records')

In [ ]:
!cat train.json

In [ ]:
!pip install seqeval

In [ ]:
!python run_ner.py \
  --model_name_or_path neuralmind/bert-base-portuguese-cased \
  --train_file ./train.json \
  --validation_file ./test.json \
  --return_entity_level_metrics \
  --output_dir ./test-ner \
  --do_train \
  --do_eval


In [ ]:
convert_to_csv_updated("test.txt")

In [ ]:
!python run_ner.py \
  --model_name_or_path ./test-ner \
  --train_file ./train.json \
  --validation_file ./train.json \
  --test_file ./test.txt.json \
  --output_dir ./test_file \
  --tokenizer_name bert-base-uncased \
  --per_device_train_batch_size 32 \
  --do_predict

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from transformers import PretrainedConfig

from transformers import TokenClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

model = AutoModelForTokenClassification.from_pretrained("./test-ner")

config = PretrainedConfig.from_pretrained("./test-ner")

pipeline = pipeline('ner', model=model, tokenizer=tokenizer, config=config)

nlp_grouped = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    grouped_entities=True
)


In [ ]:
a = nlp_grouped("SAMPLE_START RAMESWARAM : Fishermen call off strike January 05 , 2014 00:00 IST The country boat fishermen in Pam ban , who had")
a
#len(a["word"])
#a["word"]

In [ ]:
import json
with open("labels.txt", "r") as f:
  label_dict = json.loads(f.read())


submission = open("submission.txt", "w")
with open("test.txt", "r") as f:
  data = f.readlines()
  words = []
  
  for item in data:
    item = item.split()
    
  
    if not item or item[0] == "[SEP]":
      joined_words = " ".join(words)
      pred = nlp_grouped(joined_words, device=0)
      for idx, i in enumerate(pred):
        start = i["start"]
        end = i["end"]
        label_text = joined_words[start:end]
        label_text = label_text.split()
        for item in pred[idx:]
          start = i["start"]
          end = i["end"]
          label_text = joined_words[start:end]
          label_text = label_text.split()

        for token in label_text:
          submission.write(token + "\t" + label_dict[i["entity_group"]] + "\n")
      if not item:
        submission.write("\n")
      else:
        submission.write("[SEP]" + "\t" + "O\n")

       
      #print(pred)
      #submission.write(json.dumps(d) + "\n")
      words = []
    else:
      words.append(item[0])



In [ ]:
nlp_grouped("People associated with the agitation and others could also submit their suggestions")

In [ ]:
#cat en-train.txt | cut -f 2 | grep -v "^$"| sort | uniq > labels.txt
